In [4]:
import pandas as pd
import numpy as np


pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 100

import warnings
warnings.filterwarnings('ignore')

from tabula import read_pdf
# import camelot

### LDO 2018-2022

In [78]:
dd = read_pdf('../../files/loa_2022.pdf', pages='130-134' ,multiple_tables=True, encoding='utf-8', lattice=True)

In [79]:
df = pd.DataFrame()

for i in range(len(dd)):
    df = pd.concat([df,dd[i]], axis=0)
    
df = df.reset_index(drop=True)

In [35]:
# df_ff.to_excel('ldo_2.xlsx', index=False, encoding='utf-8')

### LDO 4

In [47]:
### test - '457-461'
### full - '457-1024'

dd = read_pdf('../../files/loa_2022.pdf', pages='457-1024' ,multiple_tables=True, lattice=True)

In [48]:
df = pd.DataFrame()

for i in range(len(dd)):
    ddf = dd[i]
    first_row = pd.DataFrame(ddf.columns.tolist()).T
    first_row.columns = [i for i in range(len(first_row.columns))]
    ddf.columns = [i for i in range(len(ddf.columns))]
    ddf = pd.concat([first_row,ddf],0)
    
    df = pd.concat([df,ddf], axis=0)
df = df.reset_index(drop=True)
df['aux']=np.nan

In [49]:
mask = df[0].str.contains('ÓRGÃO:').replace(np.nan,False)

df_orgao = df[mask]
df_orgao['aux'] = "org"

mask = df[0].str.contains('UNIDADE:').replace(np.nan,False)

df_unidade = df[mask]
df_unidade['aux']='uni'

mask = df[0].str.contains('PROGRAMAÇÃO DA UNIDADE ORÇAMENTÁRIA').replace(np.nan,False)

df_programacao = df[mask]
df_programacao['aux']='prog'

# indexes = list(df_programas.index)

In [50]:
df_include = pd.concat([df_orgao, df_unidade,df_programacao], axis=0).sort_index()

includes = df_include['aux'].tolist()

l3=[]
for i in range(len(includes)-2):
    if (includes[i]=='org') & (includes[i+1]=='uni') & (includes[i+2]=='prog'):
        l3.append(1)
    else:
        l3.append(0)
l3.append(1)
l3.append(1)

l4=[]        
for i in range(len(includes)-2):       
    if (includes[i-1]=='org') & (includes[i]=='uni') & (includes[i+1]=='prog'):
        l4.append(1)
    else:
        l4.append(0)
l4.append(1)
l4.append(1)

l5=[]
for i in range(len(includes)-2):       
    if (includes[i-2]=='org') & (includes[i-1]=='uni') & (includes[i]=='prog'):
        l5.append(1)
    else:
        l5.append(0)    
l5.append(1)
l5.append(1) 

df_include[3]=l3
df_include[4]=l4
df_include[5]=l5

df_include[6] = df_include[[3,4,5]].any(axis=1)
df_include[1]=df_include[2]
df_include[2]=df_include[6]
df_include = df_include.drop(columns=[3,4,5,6,7])

In [51]:
indexes = df_include.index.tolist()
ranges = df_include[2].tolist()

exclude_ranges=[]
for i in range(len(ranges)-1):
    if (ranges[i]==False) & (ranges[i+1]==True):
        exclude_ranges.append([indexes[i],indexes[i+1]])
    elif (ranges[i]==False) & (ranges[i+1]==False):
        exclude_ranges.append([indexes[i],indexes[i+1]])

In [52]:
df2 = df.copy()

for exclude in exclude_ranges:
    print(exclude[0],exclude[1])
    df2.drop(df.index[exclude[0]:exclude[1]], inplace=True)

3 104
218 282
355 448
574 622
688 774
892 973
1087 1347
1347 1612
2840 3306
3306 3697
5078 5563
5563 5776
7255 7465
7465 7631
7983 8203
8203 8404
8709 8988
8988 9086
9617 9919
9919 10033
10663 11010
11010 11381
12120 12336
12336 12577
13131 13244
13244 13364
13598 13775
13775 13939
14167 14807
14807 15300
16609 16661
16661 16712
16773 16959
16959 17108
17555 17792
17792 18000
18548 18898
18898 19135
19664 19845
19845 20023
20804 20894
20894 20973
21076 21208
21208 21307
21444 21567
21567 21674
21831 21906
21906 21971
22059 22169
22169 22269
22432 22791
22791 22991
23712 23750
23750 23784
23821 24147
24147 24373
24908 24937
24937 24967
25006 25078
25078 25128
25128 25179
25179 25231
25231 25403
25403 25480
25480 25536
25536 25692


In [53]:
mask = df2[0].str.contains('ÓRGÃO:').replace(np.nan,False)
df_orgao = df2[mask]
df_orgao=df_orgao[[0]]
# df_orgao[1] = "org"

mask = df2[0].str.contains('UNIDADE:').replace(np.nan,False)
df_unidade = df2[mask]
df_unidade=df_unidade[[0]]
df_unidade[1]='uni'

mask = df2[0].str.contains('PROGRAMAÇÃO DA UNIDADE ORÇAMENTÁRIA').replace(np.nan,False)
df_programacao = df2[mask]
df_programacao=df_programacao[[0]]
df_programacao[1]='prog'

# indexes = list(df_programas.index)

In [54]:
indexes = df_orgao.index.tolist()
orgs = df_orgao[0].tolist()
unidades = df_unidade[0].tolist()

df_aux = pd.DataFrame([1])
df_final = pd.DataFrame()

for i in range(len(unidades)):   
    
#     print(unidades[i])

    if i == len(indexes)-1:
        df_f = df2.loc[indexes[i]:]
        mask = df_f[0].str.contains('PROGRAMA:').replace(np.nan,False)
        programas = df_f[mask][0].tolist()
        prog_index = df_f[mask][0].index.tolist()
        
        
        for j in range(len(programas)):
#             print(programas[j])
            
            if j == len(programas)-1:
                #separa projetos
                df_ff = df_f.loc[prog_index[j]:]    
                mask = df_ff[0].str.contains('AÇÃO', regex=True).replace(np.nan,False)
                acoes = df_ff[mask]
                acoes['leng'] = acoes[0].apply(lambda x: len(x))
                mask = acoes['leng']==4
                acoes = acoes[mask]
                acoes_index = acoes[mask][0].index.tolist()
#                 print(len(acoes_index))
                for k in range(len(acoes_index)):
                    #separa as açoes
                    
                    if k == len(acoes)-1:
                        df_acao = df_ff.loc[acoes_index[k]:]
                        df_acao['orgaos'] = orgs[i]
                        df_acao['unidade'] = unidades[i]
                        df_acao['programas'] = programas[j]
                        df_acao['numero de acoes'] = len(acoes_index)
                        df_final = pd.concat([df_final,df_acao], axis=0)

                    else:
                        
                        df_acao = df_ff.loc[acoes_index[k]:acoes_index[k+1]-1]
                        df_acao['orgaos'] = orgs[i]
                        df_acao['unidade'] = unidades[i]
                        df_acao['programas'] = programas[j]
                        df_acao['numero de acoes'] = len(acoes_index)
                        df_final = pd.concat([df_final,df_acao], axis=0)

                
            else:
                #separa projetos
                df_ff = df_f.loc[prog_index[j]:prog_index[j+1]-1]
                mask = df_ff[0].str.contains('AÇÃO', regex=True).replace(np.nan,False)
                acoes = df_ff[mask]
                mask = df_ff[0]
                acoes['leng'] = acoes[0].apply(lambda x: len(x))
                mask = acoes['leng']==4
                acoes = acoes[mask]
                acoes_index = acoes[mask][0].index.tolist()
#                 print(len(acoes_index))
                for k in range(len(acoes_index)):
                    #separa as açoes
                    
                    if k == len(acoes)-1:
                        df_acao = df_ff.loc[acoes_index[k]:]
                        df_acao['orgaos'] = orgs[i]
                        df_acao['unidade'] = unidades[i]
                        df_acao['programas'] = programas[j]
                        df_acao['numero de acoes'] = len(acoes_index)
                        df_final = pd.concat([df_final,df_acao], axis=0)

                    else:
                        
                        df_acao = df_ff.loc[acoes_index[k]:acoes_index[k+1]-1]
                        df_acao['orgaos'] = orgs[i]
                        df_acao['unidade'] = unidades[i]
                        df_acao['programas'] = programas[j]
                        df_acao['numero de acoes'] = len(acoes_index)
                        df_final = pd.concat([df_final,df_acao], axis=0)
                        
                        
                        
                        
                        
    else:
        
        df_f = df2.loc[indexes[i]:indexes[i+1]]
        mask = df_f[0].str.contains('PROGRAMA:').replace(np.nan,False)
        programas = df_f[mask][0].tolist()
        prog_index = df_f[mask][0].index.tolist()

        for j in range(len(programas)):
#             print(programas[j])
            if j == len(programas)-1:
                #separa projetos
                df_ff = df_f.loc[prog_index[j]:]   
#                 display(df_ff)
#                 display(df_ff)

                mask = df_ff[0].str.contains('AÇÃO', regex=True).replace(np.nan,False)
                acoes = df_ff[mask]
                acoes['leng'] = acoes[0].apply(lambda x: len(x))
                mask = acoes['leng']==4
                acoes = acoes[mask]
                acoes_index = acoes[mask][0].index.tolist()
#                 print(len(acoes_index))
                
                for k in range(len(acoes_index)):
                    #separa as açoes
                    
                    if k == len(acoes)-1:
                        df_acao = df_ff.loc[acoes_index[k]:]
                        df_acao['orgaos'] = orgs[i]
                        df_acao['unidade'] = unidades[i]
                        df_acao['programas'] = programas[j]
                        df_acao['numero de acoes'] = len(acoes_index)
                        df_final = pd.concat([df_final,df_acao], axis=0)

                    else:
                        
                        df_acao = df_ff.loc[acoes_index[k]:acoes_index[k+1]-1]
                        df_acao['orgaos'] = orgs[i]
                        df_acao['unidade'] = unidades[i]
                        df_acao['programas'] = programas[j]
                        df_acao['programas'] = programas[j]
                        df_acao['numero de acoes'] = len(acoes_index)
                        df_final = pd.concat([df_final,df_acao], axis=0)

                
            else:
                #separa projetos
                df_ff = df_f.loc[prog_index[j]:prog_index[j+1]-1]
                mask = df_ff[0].str.contains('AÇÃO', regex=True).replace(np.nan,False)
                acoes = df_ff[mask]
                mask = df_ff[0]
                acoes['leng'] = acoes[0].apply(lambda x: len(x))
                mask = acoes['leng']==4
                acoes = acoes[mask]
                acoes_index = acoes[mask][0].index.tolist()
#                 print(len(acoes_index))

                for k in range(len(acoes_index)):
                    #separa as açoes
                    
                    if k == len(acoes)-1:
                        df_acao = df_ff.loc[acoes_index[k]:]
                        df_acao['orgaos'] = orgs[i]
                        df_acao['unidade'] = unidades[i]
                        df_acao['programas'] = programas[j] 
                        df_acao['numero de acoes'] = len(acoes_index)
                        df_final = pd.concat([df_final,df_acao], axis=0)

                    else:
                        
                        df_acao = df_ff.loc[acoes_index[k]:acoes_index[k+1]-1]
                        df_acao['orgaos'] = orgs[i]
                        df_acao['unidade'] = unidades[i]
                        df_acao['programas'] = programas[j]
                        df_acao['numero de acoes'] = len(acoes_index)
                        df_final = pd.concat([df_final,df_acao], axis=0)
            
#             df_final = pd.concat([df_final,df_aux],axis=0)
            
        
#         print(indexes[i], orgs[i], unidades[i])

In [55]:
df_final.to_excel('pre_ldo_4.xlsx', index=False, encoding='utf-8')

### LDO_4 Revival

In [56]:
df = pd.read_excel('pre_ldo_4.xlsx')

In [57]:
df[3] = np.where(df[2]=='FONTE DE RECURSO', 'FONTE DE RECURSO',df[3])
df[2] = np.where(df[1]=='GRUPO DESPESA','GRUPO DESPESA',df[2])
df[1] = np.where(df[0]=='AÇÃO', 'AÇÃO',df[1])
df[0] = np.where(df[0]=='AÇÃO', 'NUMERO DA AÇÃO',df[0])

df[5] = np.where(df[0]=='PRODUTO:', df[1],np.nan)
df[6] = np.where(df[0]=='INDICADOR DE\rPRODUTO:', df[1], np.nan)
df[7] = np.where(df[0]=='META:', df[1], np.nan)


df[1] = np.where((df[0]=='PRODUTO:') | (df[0]=='INDICADOR DE\rPRODUTO:') | (df[0]=='META:'), np.nan,df[1])
df[0] = np.where((df[0]=='PRODUTO:') | (df[0]=='INDICADOR DE\rPRODUTO:') | (df[0]=='META:'), np.nan,df[0])


df[5] = np.where(df[0]=='NUMERO DA AÇÃO', 'PRODUTO',df[5])
df[6] = np.where(df[0]=='NUMERO DA AÇÃO', 'INDICADOR DE PRODUTO',df[6])
df[7] = np.where(df[0]=='NUMERO DA AÇÃO', 'META',df[7])

In [58]:
df[0].fillna(method='ffill', inplace=True)
df[1].fillna(method='ffill', inplace=True)
df[2].fillna(method='ffill', inplace=True)
df[5].fillna(method='bfill', inplace=True)
df[6].fillna(method='bfill', inplace=True)
df[7].fillna(method='bfill', inplace=True)

#remove os nulls das fontes de recurso
mask = df[3].notnull()
df = df[mask]

#remove os nulls dos valores
mask = df[4].notnull()
df = df[mask]

# coloca nan nas linhas que nao possuem meta
df[7] = np.where(df[7]=='META', np.nan, df[7])

# df = df.drop(columns=[8])

In [59]:
rename = {
    0:'codigo da acao',
    1:'acao',
    2:'grupo despesa',
    3:'fonte de recurso',
    4:'valor',
    5:'produto',
    6:'indicador de produto',
    7:'meta'
}

df = df.rename(columns=rename)

In [60]:
df.columns

Index([      'codigo da acao',                 'acao',        'grupo despesa',
           'fonte de recurso',                'valor',              'produto',
       'indicador de produto',                 'meta',                      8,
                        'aux',               'orgaos',              'unidade',
                  'programas',      'numero de acoes'],
      dtype='object')

In [61]:
cols = [
    'orgaos', 'unidade',
    'programas', 
    'codigo da acao', 'acao','numero de acoes',
    'produto','indicador de produto', 
    'grupo despesa', 'fonte de recurso', 
    'meta','valor'
]

df = df[cols]

In [62]:
mask = (df['acao']=='AÇÃO') | (df['valor'].str.contains('Unnamed'))
df = df[np.logical_not(mask)]

In [63]:
for col in df.columns:
    if (col!='numero de acoes'):
        df[col] = df[col].str.replace('\r',' ')
    
    
df['valor'] =  df['valor'].str.replace('.','').str.replace(',','.').astype(float)
df['meta']  =  df['meta'].str.replace('.','').str.replace(',','.').astype(float)

In [64]:
df['contador'] = 1

In [66]:
replace_values = {
    'UTRAS DESPESAS ORRENTES':'OUTRAS DESPESAS CORRENTES',
    'NVESTIMENTOS':'INVESTIMENTOS ',
    'ESSOAL E ENCARGOS OCIAI':'PESSOAL E ENCARGOS SOCIAIS',
    'NVERSÕES FINANCEIRAS':'INVERSÕES FINANCEIRAS',
    'ESERVA DE ONTINGÊNCIA':'RESERVA DE CONTINGÊNCIA',
    'UROS E ENCARGOS DA ÍVIDA':'JUROS E ENCARGOS DA DÍVIDA',
    'MORTIZAÇÃO DA DÍVIDA':'AMORTIZAÇÃO DA DÍVIDA'
}


df = df.replace({"grupo despesa": replace_values})

In [67]:
replace_values = {
    'ESOURO DO ESTADO':'TESOURO DO ESTADO',
    'INCULADOS FEDERAIS':'VINCULADOS FEDERAIS',
    'UNDO ESPECIAL DE ESPESA':'FUNDO ESPECIAL DE DESPESA',
    'RÓPRIOS':'PRÓPRIOS',
    'INCULADOS ESTADUAIS':'VINCULADOS ESTADUAIS',
    'UTRAS FONTES - UNDOS':'OUTRAS FONTES - FUNDOS',
    'ESOURO-DOT.INICIAL E RED.SUPLEMENTAR- NTRA':'TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR- INTRA',
    'PERAÇÕES DE RÉDITO':'OPERAÇÕES DE CRÉDITO',
    'UTRAS FONTES - DREM':'OUTRAS FONTES - DREM',
    'ECURSOS INCULADOS-FUNDO SP. DESPESA-INTRA':'RECURSOS VINCULADOS-FUNDO ESP. DESPESA-INTRA',
    'ECURSOS VINCULADOS EDERAIS-INTRA':'RECURSOS VINCULADOS FEDERAIS-INTRA',
    'EC.PROPRIO-ADM.IND.- OT.INIC.CR.SUPL.- NTRA':'REC.PROPRIO-ADM.IND.- DOT.INIC.CR.SUPL.- INTRA',
    'P.CRED.CONTR.EXTERI R-DOT.INIC.CR.SUP- NTRA':'OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA',
    'UTRAS FONTES DE ECURSOS-INTRA':'OUTRAS FONTES DE RECURSOS-INTRA '
    
}


df = df.replace({"fonte de recurso": replace_values})

In [68]:
# df['indicador de produto'] = df['indicador de produto'].str.replace('ERCENTUAL','PERCENTUAL').str.replace('ÚMERO','NÚMERO').str.replace('XTENSÃO','EXTENSÃO').str.replace('ENEFICIÁRIOS','BENEFICIÁRIOS')

# df['indicador de produto'] = df['indicador de produto'].str.replace('ERCENTUAL','PERCENTUAL').str.replace('ÚMERO','NÚMERO')

In [69]:
# mask = (df['valor']==32265100)&(df['codigo da acao']=='roposta Orçamentária 2020')

# df = df.set_value(df[mask].index,'codigo da acao' ,'10.844.0000.9019')
# df = df.set_value(df[mask].index,'acao' ,'PAGAMENTO DA DÍVIDA PÚBLICA EXTERNA')

# mask = (df['valor']==51720442)&(df['codigo da acao']=='roposta Orçamentária 2020')

# df = df.set_value(df[mask].index,'codigo da acao' ,'10.844.0000.9019')
# df = df.set_value(df[mask].index,'acao' ,'PAGAMENTO DA DÍVIDA PÚBLICA EXTERNA')



# mask = (df['valor']==28623000)&(df['codigo da acao']=='roposta Orçamentária 2020')

# df = df.set_value(df[mask].index,'codigo da acao' ,'10.303.0935.4192')
# df = df.set_value(df[mask].index,'acao' ,'ATENDIMENTO HEMOTERÁPICO')




# mask = (df['programas']=='PROGRAMA:     0930    ATENDIMENTO INTEGRAL E DESCENTRALIZADO NO SUS NO ESTADO DE SÃO PAULO') & (df['codigo da acao']=='roposta Orçamentária 2020')

# df = df.set_value(df[mask].index,'codigo da acao' ,'10.302.0930.5274')
# df = df.set_value(df[mask].index,'acao' ,'ASSISTÊNCIA MÉDICA, HOSPITALAR E AMBULATORIAL')



# mask = (df['valor']==2202907)&(df['codigo da acao']=='roposta Orçamentária 2020')

# df = df.set_value(df[mask].index,'codigo da acao' ,'18.542.2604.2622')
# df = df.set_value(df[mask].index,'acao' ,'CONTRAPARTIDAS AMBIENTAIS E CONVÊNIOS')



In [70]:
# df.groupby(by='acao').sum().sort_values(by='valor',ascending=False)

In [71]:
# df.head()

In [72]:
df.groupby(by='unidade').sum()

,numero de acoes,meta,valor,contador
unidade,,,,
UNIDADE: 01001 - ASSEMBLEIA LEGISLATIVA,126,5.626200e+04,1.287450e+09,14
UNIDADE: 02001 - TRIBUNAL DE CONTAS DO ESTADO,32,9.218000e+03,1.082664e+09,8
UNIDADE: 03001 - TRIBUNAL DE JUSTIÇA,216,5.629333e+06,1.352414e+10,27
UNIDADE: 06001 - TRIBUNAL DE JUSTIÇA MILITAR,21,1.090500e+04,7.315575e+07,7
UNIDADE: 08001 - ADMINISTRAÇÃO SUPERIOR DA SECRETARIA E SEDE,527,2.852347e+07,2.710854e+10,56
UNIDADE: 08002 - CONSELHO ESTADUAL DE EDUCAÇÃO-CEE,5,0.000000e+00,1.848493e+06,5
UNIDADE: 08009 - COORDENADORIA GESTÃO DE RECURSOS HUMANOS,39,2.400320e+05,5.045937e+08,13
UNIDADE: 08010 - ESCOLA FORMAÇÃO PROF.ED.EST.PAULO R.C.SOUZA,41,1.200320e+05,3.774987e+07,15
UNIDADE: 08011 - COORDENADORIA PEDAGÓGICA,183,4.519823e+06,6.435884e+08,29


In [76]:
df.to_excel('../../data/loa/loa_2022/loa_2022.xlsx', index=False)

### LDO 5

In [262]:
### test - '339-396'
### full - '846-869'

dd = read_pdf('files/ldo_full.pdf', pages='846-869' ,multiple_tables=True, encoding='utf-8', lattice=True)

In [263]:
df = pd.DataFrame()

for i in range(len(dd)):
    df = pd.concat([df,dd[i]], axis=0)
    
df = df.reset_index(drop=True)

In [264]:
mask = df[0].str.contains('ÓRGÃO:').replace(np.nan,False)

df_orgao = df[mask]
df_orgao[2] = "org"

mask = df[0].str.contains('EMPRESA:').replace(np.nan,False)

df_empresa = df[mask]
df_empresa[2]='uni'

mask = df[0].str.contains('DEMONSTRATIVO DOS INVESTIMENTOS').replace(np.nan,False)

df_demonstrativo = df[mask]
df_demonstrativo[2]='dem'

# indexes = list(df_programas.index)

In [265]:
df_exclude = pd.concat([df_empresa,df_demonstrativo],axis=0).sort_index()
exclude_list=[df_exclude.index.tolist()[i:i+2] for i in range(0,len(df_exclude.index.tolist()),2)]

In [266]:
df2 = df.copy()

for exclude in exclude_list:
    print(exclude[0],exclude[1])
    df2.drop(df.index[exclude[0]+1:exclude[1]+1], inplace=True)

6 38
75 107
148 178
198 229
249 280
301 338
449 483
527 560
583 617
731 762
877 910
942 974
1006 1037


In [267]:
df_f = df2.copy()

df_f['orgao'] = np.where(df_f[0]=='ÓRGÃO:',df_f[0]+" "+df_f[1], np.nan)
df_f[1] = np.where(df_f[0]=='ÓRGÃO:',np.nan,df_f[1])
df_f[0] = np.where(df_f[0]=='ÓRGÃO:',np.nan,df_f[0])

df_f['empresa'] = np.where(df_f[0]=='EMPRESA:',df_f[0]+" "+df_f[1], np.nan)
df_f[1] = np.where(df_f[0]=='EMPRESA:',np.nan,df_f[1])
df_f[0] = np.where(df_f[0]=='EMPRESA:',np.nan,df_f[0])

df_f['programa'] = np.where(df_f[0].str.contains('PROGRAMA:'),df_f[0]+" "+df_f[1], np.nan)
df_f[1] = np.where((df_f[0].str.contains('PROGRAMA:') & df_f[0].notnull()),np.nan,df_f[1])
df_f[0] = np.where((df_f[0].str.contains('PROGRAMA:') & df_f[0].notnull()),np.nan,df_f[0])

df_f['produto'] = np.where(df_f[0]=='PRODUTO:',df_f[0]+" "+df_f[1], np.nan)
df_f[1] = np.where(df_f[0]=='PRODUTO:',np.nan,df_f[1])
df_f[0] = np.where(df_f[0]=='PRODUTO:',np.nan,df_f[0])


df_f['indicador de produto'] = np.where(df_f[0]=='INDICADOR\rDE PRODUTO:',df_f[0]+" "+df_f[1], np.nan)
df_f[1] = np.where(df_f[0]=='INDICADOR\rDE PRODUTO:',np.nan,df_f[1])
df_f[0] = np.where(df_f[0]=='INDICADOR\rDE PRODUTO:',np.nan,df_f[0])


df_f['descricao'] = np.where(df_f[0]=='DESCRIÇÃO:',df_f[0]+" "+df_f[1], np.nan)
df_f[1] = np.where(df_f[0]=='DESCRIÇÃO:',np.nan,df_f[1])
df_f[0] = np.where(df_f[0]=='DESCRIÇÃO:',np.nan,df_f[0])


df_f[2] = df_f[2].str.replace('.','')
df_f[2] = df_f[2].fillna('aaaaaaaaaa')

mask = df_f[2].str.isdigit()
df_f['valor'] = np.where(mask, df_f[2],np.nan)
df_f[2] = np.where(mask, np.nan, df_f[2])

df_f[2] = np.where(df_f[2]=='aaaaaaaaaa', np.nan,df_f[2])
df_f[2] = np.where(df_f[2]=='Valores em R$ 1,00', np.nan,df_f[2])

df_f['acao'] = df_f[2]

df_f = df_f.drop(columns=[2])


mask = df_f[0].apply(lambda x: str(x)[:2].isdigit())
df_f['numero da acao'] = np.where(mask, df_f[0], np.nan)
df_f[0] = np.where(mask, np.nan, df_f[0])


mask = df_f[0]=='OBJETIVO'
objetivo = [i+1 for i in df_f[mask].index.tolist()]
df_f['objetivo'] = 'aaaaaaaaaa'

for i in objetivo:
    df_f = df_f.set_value(i,'objetivo' ,df_f.loc[i][0])
df_f['objetivo'] = np.where(df_f['objetivo']=='aaaaaaaaaa', np.nan,df_f['objetivo'])

    
mask = df_f[1]=='PÚBLICO ALVO'
publico = [i+1 for i in df_f[mask].index.tolist()]
df_f['publico alvo'] = 'aaaaaaaaaa'

for i in publico:
    df_f = df_f.set_value(i,'publico alvo' ,df_f.loc[i][1])
    df_f = df_f.set_value(i,1 ,np.nan)
    df_f = df_f.set_value(i-1,1 ,np.nan)
    
    
df_f['publico alvo'] = np.where(df_f['publico alvo']=='aaaaaaaaaa', np.nan,df_f['publico alvo'])

mask =  (df_f[1].notnull()) & (df_f['valor'].notnull())
df_f['fonte de financiamento'] = np.where(mask, df_f[1],np.nan)


In [270]:
df_f['orgao'].fillna(method='ffill', inplace=True)
df_f['empresa'].fillna(method='ffill', inplace=True)
df_f['programa'].fillna(method='ffill', inplace=True)
df_f['produto'].fillna(method='bfill', inplace=True)
df_f['indicador de produto'].fillna(method='bfill', inplace=True)
df_f['descricao'].fillna(method='bfill', inplace=True)
df_f['acao'].fillna(method='ffill', inplace=True)
df_f['numero da acao'].fillna(method='bfill', inplace=True)
df_f['objetivo'].fillna(method='bfill', inplace=True)
df_f['publico alvo'].fillna(method='bfill', inplace=True)


mask = df_f['fonte de financiamento'].notna()

df_f = df_f[mask].drop(columns = [0,1,3,4,5])

df_f['valor'] = df_f['valor'].astype(float)




In [277]:
df_f.columns

Index(['orgao', 'empresa', 'programa', 'produto', 'indicador de produto',
       'descricao', 'valor', 'acao', 'numero da acao', 'objetivo',
       'publico alvo', 'fonte de financiamento'],
      dtype='object')

In [280]:
cols = [
    'orgao', 'empresa', 'programa', 
    'produto','indicador de produto', 'objetivo', 'descricao',  'publico alvo',
    'numero da acao','acao', 'fonte de financiamento', 'valor'
]

df_f = df_f[cols]


for col in df_f.columns:
    
    if col!='valor':
        df_f[col] = df_f[col].str.replace('\r',' ')

In [283]:
# df_f.to_excel('ldo_5.xlsx', index=False, encoding='utf-8')

In [284]:
df_f.groupby(by='programa').sum()

,valor
programa,
PROGRAMA:1606 MELHORIAS DA MALHA RODOVIÁRIA,1.000000e+01
PROGRAMA:1608 TRAVESSIAS LITORÂNEAS,1.000000e+01
PROGRAMA:2005 FOMENTO AO DESENVOLVIMENTO SOCIOECONÔMICO,2.571072e+08
PROGRAMA:2027 GESTÃO DE SERVIÇOS PRESTADOS À ADMINISTRAÇÃO ESTADUAL,1.000000e+01
PROGRAMA:2028 GESTÃO INOVADORA E ORIENTADA PARA RESULTADOS,1.000000e+01
PROGRAMA:2505 FOMENTO À HABITAÇÃO DE INTERESSE SOCIAL- CASA PAULISTA,7.275540e+06
PROGRAMA:2507 REGULARIZAÇÃO FUNDIÁRIA DE INTERESSE HABITACIONAL,6.043646e+07
PROGRAMA:2508 PROVISÃO DE MORADIAS,6.377098e+08
PROGRAMA:2510 REQUALIFICAÇÃO HABITACIONAL E URBANA,4.846704e+08


### LOA 2019 EMENDAS SAUDE

In [60]:
dd = camelot.read_pdf('files/loa_2019.pdf', pages='22-52')

In [61]:
df = pd.DataFrame()

for i in range(len(dd)):
    df = pd.concat([df,dd[i].df], axis=0)
    
df = df.reset_index(drop=True)

In [76]:
df2 = df.replace('\r',' ', regex=True).replace('\n',' ', regex=True)

In [77]:
mask = df[0]!='DEPUTADO (A)'
df2 = df2[mask].rename(columns={0:'DEPUTADO (A)'})

mask = df[1]!='ENTIDADE / PREFEITURA \nBENEFICIADA'
df2 = df2[mask].rename(columns={1:'ENTIDADE / PREFEITURA \nBENEFICIADA'})

mask = df[2]!='MUNICÍPIO'
df2 = df2[mask].rename(columns={2:'MUNICÍPIO'})

mask = df[3]!='CNPJ'
df2 = df2[mask].rename(columns={3:'CNPJ'})

mask = df[4]!='OBJETO'
df2 = df2[mask].rename(columns={4:'OBJETO'})

mask = df[5]!='VALOR'
df2 = df2[mask].rename(columns={5:'VALOR'})

df2['VALOR'] = df2['VALOR'].str.replace('.','')

df2['VALOR'] = pd.to_numeric(df2['VALOR'],errors='coerce')


In [92]:
# df2.to_excel('ldo_2019_emendas_saude.xlsx', index=False)

### LOA 2019 EMENDAS EXCETO SAUDE

In [93]:
dd = camelot.read_pdf('files/loa_2019.pdf', pages='53-95')

In [94]:
df = pd.DataFrame()

for i in range(len(dd)):
    df = pd.concat([df,dd[i].df], axis=0)
    
df = df.reset_index(drop=True)

In [100]:
df2 = df.replace('\r',' ', regex=True).replace('\n',' ', regex=True)

In [101]:
mask = df[0]!='DEPUTADO (A)'
df2 = df2[mask].rename(columns={0:'DEPUTADO (A)'})

mask = df[1]!='ENTIDADE / PREFEITURA \nBENEFICIADA'
df2 = df2[mask].rename(columns={1:'ENTIDADE / PREFEITURA \nBENEFICIADA'})

mask = df[2]!='MUNICÍPIO'
df2 = df2[mask].rename(columns={2:'MUNICÍPIO'})

mask = df[3]!='CNPJ'
df2 = df2[mask].rename(columns={3:'CNPJ'})

mask = df[4]!='OBJETO'
df2 = df2[mask].rename(columns={4:'OBJETO'})

mask = df[5]!='SECRETARIA / \nÓRGÃO'
df2 = df2[mask].rename(columns={5:'SECRETARIA / \nÓRGÃO'})

mask = df[6]!='VALOR'
df2 = df2[mask].rename(columns={6:'VALOR'})

df2['VALOR'] = df2['VALOR'].str.replace('.','')

df2['VALOR'] = pd.to_numeric(df2['VALOR'],errors='coerce')

In [102]:
df2.head()

,DEPUTADO (A),ENTIDADE / PREFEITURA BENEFICIADA,MUNICÍPIO,CNPJ,OBJETO,SECRETARIA / ÓRGÃO,VALOR
1,ABELARDO CAMARINHA,PREFEITURA MUNICIPAL DE BASTOS,BASTOS,45.547.403/0001-93,PLANEJAMENTO,INFRAESTRUTURA,250000.0
2,ABELARDO CAMARINHA,PREFEITURA MUNICIPAL DE FLÓRIDA PAULISTA,FLÓRIDA PAULISTA,44.925.691/0001-00,PLANEJAMENTO,INFRAESTRUTURA,200000.0
3,ABELARDO CAMARINHA,PREFEITURA MUNICIPAL DE GÁLIA,GÁLIA,44.518.389/0001-37,PLANEJAMENTO,INFRAESTRUTURA,250000.0
4,ABELARDO CAMARINHA,PREFEITURA MUNICIPAL DE GARÇA,GARÇA,44.518.371/0001-35,PLANEJAMENTO,INFRAESTRUTURA,100000.0
5,ABELARDO CAMARINHA,PREFEITURA MUNICIPAL DE GETULINA,GETULINA,44.528.842/0001-96,PLANEJAMENTO,INFRAESTRUTURA,100000.0


In [108]:
# df2.to_excel('ldo_2019_emendas_exceto_saude.xlsx', index=False)

In [109]:
df = pd.read_excel('2019/ldo_2019_emendas_exceto_saude.xlsx')

In [115]:
df.groupby(by=['DEPUTADO (A)']).sum().sort_values(by='VALOR', ascending=False)

,VALOR
DEPUTADO (A),
ABELARDO CAMARINHA,2453970
JOÃO PAULO RILLO,2453970
MARIA LÚCIA AMARY,2453970
MARCOS ZERBINI,2453970
MARCOS MARTINS,2453970
MARCOS DAMASIO,2453970
MARCO VINHOLI,2453970
MARCIO CAMARGO,2453970
LUIZ CARLOS GONDIM,2453970


In [126]:
df.groupby(by=['OBJETO']).sum().sort_values(by='VALOR', ascending=False).head(10)

,VALOR
OBJETO,
PLANEJAMENTO,76020220
DESENVOLVIMENTO SOCIAL,34544610
PLANEJAMENTO E GESTÃO,29509260
SAÚDE,12108880
SECRETARIA DE PLANEJAMENTO E GESTÃO,9484940
*,9361880
CULTURA,6125910
SECRETARIA DE DESENVOLVIMENTO SOCIAL,5962970
EDUCAÇÃO,4220000


In [127]:
df.sort_values(by='VALOR', ascending=False).head(10)

,DEPUTADO (A),ENTIDADE / PREFEITURA BENEFICIADA,MUNICÍPIO,CNPJ,OBJETO,SECRETARIA / ÓRGÃO,VALOR
87,ANALICE FERNANDES,*,*,*,PLANEJAMENTO,*,2453970
654,GILMACI SANTOS,*,*,*,*,*,2453970
99,ANDRÉ SOARES,PREFEITURA DE SÃO PAULO,SÃO PAULO,46.395.000/0001-39,SAÚDE,AQUISIÇÃO DE EQUIPAMENTOS,2453970
1058,MILTON VIEIRA,*,*,*,*,*,2453970
553,FELICIANO FILHO,PREFEITURA MUNICIPAL,GUARULHOS,46.319.000/0001-50,SAÚDE,DESPESAS DE CUSTEIO,2453970
179,CARLÃO PIGNATARI,*,*,*,PLANEJAMENTO,*,2453970
774,JOOJI HATO,*,*,*,*,*,2453970
259,CAUÊ MACRIS,*,*,*,PLANEJAMENTO,*,2453970
401,DELEGADO OLIM,*,*,*,NaN,*,2453970
1185,REINALDO ALGUZ,*,*,*,PLANEJAMENTO,*,2378970


In [122]:
df.columns

Index(['DEPUTADO (A)', 'ENTIDADE / PREFEITURA \nBENEFICIADA', 'MUNICÍPIO',
       'CNPJ', 'OBJETO', 'SECRETARIA / \nÓRGÃO', 'VALOR'],
      dtype='object')